In [18]:
def salvar_html(html, filename):
    """Salva o resultado de uma requisição para visualização (sem css) no browser
    """
    with open(filename,'w') as myf:
        myf.write(html)
    

In [19]:
import mechanize
from urllib import urlencode
from bs4 import BeautifulSoup as bsoup
import sqlite3
#Browser
driver = mechanize.Browser()
clear_cookies = driver._ua_handlers['_cookies'].cookiejar.clear
#URLs
base_url = "https://www.ncbi.nlm.nih.gov"

### Encontrando os artigos na primeira página

In [24]:
### Pesquisar
pesquisa = {'term':'shy'}
resposta = driver.open(base_url + '/pmc/', urlencode(pesquisa))

### Pegar resultados
soup = bsoup(resposta.read(), "html5lib")
#Título da página
print soup.findAll('title')[0].text
# Número da página
print soup.findAll('input',id='pageno')[0]['value']
#Títulos dos papers encontrados
links = soup.findAll("div", {"class":"title"})
print '\n\n'.join([link.text for link in links[:3]])

shy - PMC - NCBI
1
PMP22 exon 4 deletion causes ER retention of PMP22 and a gain‐of‐function allele in CMT1E

Charcot Marie Tooth (CMT) Subtypes and Genetic Testing Strategies

High-Dosage Ascorbic Acid Treatment in Charcot-Marie-Tooth Disease Type 1A: Results of a Randomized, Double-Masked, Controlled Trial


### Encontrando os artigos depois da primeira página

In [25]:
from imports_pmc import _querystring, _pageno
### Pesquisar, setando o request com o número da página
_querystring[_pageno] = 3
resposta = driver.open("https://www.ncbi.nlm.nih.gov/pmc/", urlencode(_querystring))

### Pegar resultados
soup = bsoup(resposta.read(), "html5lib")
#Título da página
print soup.findAll('title')[0].text
# Número da página
print soup.findAll('input',id='pageno')[0]['value']
#Títulos dos papers encontrados
links = soup.findAll("div", {"class":"title"})
print '\n\n'.join([link.text for link in links[:3]])

shy - PMC - NCBI
3
Validation of the CMT Pediatric Scale as an outcome measure of disability

Reduced neurofilament expression in cutaneous nerve fibers of patients with CMT2E

Small nerve fiber involvement in CMT1A


In [28]:
[link.findAll('a', href=True)[0]['href'] for link in links[:3]]

[u'/pmc/articles/PMC3335189/',
 u'/pmc/articles/PMC4516296/',
 u'/pmc/articles/PMC4336000/']